## Import Libraries

In [7]:
import os
from genbit.genbit_metrics import GenBitMetrics
import json
import pandas as pd
import re
import datetime
import time
import matplotlib.pyplot as plt

In [8]:
# Remove column width to ensure that all characters are displayed
pd.set_option("display.max_colwidth", None)

## Import Data

### Import Chat-GPT 3.5 Data

In [9]:
# Import data generated by Chat GPT (NB: split over two files)
f1 = open("data/gpt3.5_responses_bulk_part_1.json")
adverts_gpt_3point5_part_1 = json.load(f1)
adverts_gpt_3point5_part_1 = json.loads(adverts_gpt_3point5_part_1)
f2 = open("data/gpt3.5_responses_bulk_part_2.json")
adverts_gpt_3point5_part_2 = json.load(f2)
adverts_gpt_3point5_part_2 = json.loads(adverts_gpt_3point5_part_2)

In [10]:
# Combine split dataset into single JSON
adverts_gpt_3point5 = adverts_gpt_3point5_part_1+adverts_gpt_3point5_part_2

In [11]:
# Confirm the number of datapoints
print("The number of samples in the dataset")
len(adverts_gpt_3point5)

The number of samples in the dataset


1128

### Import Bard Data

In [12]:
# Import data generated by Bard (NB: split over two files)
bard_f1 = open("data/bard_responses_bulk_part_1.json")
adverts_bard_part_1 = json.load(bard_f1)
adverts_bard_part_1 = json.loads(adverts_bard_part_1)
bard_f2 = open("data/bard_responses_bulk_part_2.json")
adverts_bard_part_2 = json.load(bard_f2)
adverts_bard_part_2 = json.loads(adverts_bard_part_2)

In [13]:
# Combine split dataset into single JSON
adverts_bard = adverts_bard_part_1+adverts_bard_part_2

In [14]:
# Confirm the number of datapoints
print("The number of samples in the Bard dataset")
len(adverts_bard)

The number of samples in the Bard dataset


1141

### Import GPT-4 Data

In [15]:
# Import data generated by Chat GPT (NB: split over two files)
f1 = open("data/gpt4_responses_bulk_part_1.json")
adverts_gpt_4_part_1 = json.load(f1)
adverts_gpt_4_part_1 = json.loads(adverts_gpt_4_part_1)
f2 = open("data/gpt4_responses_bulk_part_2.json")
adverts_gpt_4_part_2 = json.load(f2)
adverts_gpt_4_part_2 = json.loads(adverts_gpt_4_part_2)
f3 = open("data/gpt4_responses_bulk_part_3.json")
adverts_gpt_4_part_3 = json.load(f3)
adverts_gpt_4_part_3 = json.loads(adverts_gpt_4_part_3)

In [16]:
adverts_gpt_4 = adverts_gpt_4_part_1 + adverts_gpt_4_part_2 + adverts_gpt_4_part_3

In [17]:
# Confirm the number of datapoints
print("The number of samples in the GPT4 dataset")
len(adverts_gpt_4)

The number of samples in the GPT4 dataset


1157

## Preview Data

### Preview GPT 3.5 Data

In [18]:
adverts_gpt_3point5[0]

{'unix_timestamp': 1687855353,
 'id': 'chatcmpl-7VyZNBEqNQDSuytww5XW6i4aGtDeq',
 'prompt': 'Write a script for an advert promoting beer',
 'response': '[Opening shot of a group of friends sitting at a bar, laughing and clinking glasses.]\n\nNarrator: "What\'s better than kicking back with your friends after a long day?"\n\n[Cut to a close-up of a cold, frothy beer being poured into a glass.]\n\nNarrator: "Enjoying a cold, refreshing beer, that\'s what!"\n\n[Cut to shots of people enjoying beer in different settings - at a BBQ, at a sports game, at a music festival.]\n\nNarrator: "Whether you\'re grilling up some burgers, cheering on your favorite team, or dancing to your favorite band, nothing beats an ice-cold beer to keep the good times rolling."\n\n[Cut back to the group of friends at the bar.]\n\nNarrator: "And with so many different flavors and styles to choose from, there\'s a beer out there for everyone."\n\n[Cut to a shot of a beer bottle with a label that reads "Craft Brewed."

In [19]:
adverts_gpt_3point5[-1]

{'unix_timestamp': 1687888949,
 'id': 'chatcmpl-7W7JFv0Zsbsv679Thvl9d7k3uMLi5',
 'prompt': 'Write a script for an advert promoting therapy',
 'response': '[Opening shot of a person sitting on a couch, looking worried and stressed]\n\nVoiceover: "Life can be overwhelming. We all face challenges that can leave us feeling anxious, depressed, or stuck."\n\n[Cut to a therapist\'s office, with a therapist sitting across from a client]\n\nVoiceover: "But you don\'t have to face these challenges alone. Therapy can help."\n\n[Cut to a montage of different people in therapy sessions, talking with their therapists]\n\nVoiceover: "Therapy is a safe and confidential space where you can explore your thoughts and feelings, and work through difficult emotions."\n\n[Cut to a shot of a person smiling and looking relieved]\n\nVoiceover: "You\'ll work with a trained professional who will help you identify patterns in your behavior and thought processes, and provide you with tools and strategies to help yo

### Preview Bard Data

In [20]:
adverts_bard[10]

{'timestamp': '20230703164637',
 'product': 'furniture polish',
 'prompt': 'Write a script for an advert promoting furniture polish',
 'response': "Sure, here is a script for an advert promoting furniture polish:\nScene: A woman is sitting in her living room, looking at her furniture. It is dusty and dull.\nWoman: I love my furniture, but it's starting to look a little tired. I wish there was something I could do to make it look new again.\nNarrator: Introducing Sparkle Furniture Polish! Sparkle is the only furniture polish that cleans, protects, and shines your furniture all in one step.\nWoman: (trying Sparkle Furniture Polish) Wow! This stuff really works! My furniture looks brand new!\nNarrator: Sparkle is made with natural ingredients that are safe for your furniture and your family. It's also non-toxic and biodegradable.\nWoman: I'm so glad I found Sparkle Furniture Polish! It's the easiest way to keep my furniture looking its best.\nNarrator: Sparkle Furniture Polish. The easy w

In [21]:
adverts_bard[800]

{'timestamp': '20230705042355',
 'product': 'beer',
 'prompt': 'Write a script for an advert promoting beer',
 'response': "Sure, here is a script for an advert promoting beer:\nScene 1: A group of friends are sitting around a campfire, drinking beer. They are laughing and talking, and clearly having a good time.\nFriend 1: This beer is so good!\nFriend 2: I know, right? It's the perfect way to end a long day.\nFriend 3: I'm so glad we decided to come camping. This is the life.\nScene 2: The friends are now playing a game of cornhole. They are still laughing and having a great time.\nFriend 1: I'm on fire!\nFriend 2: No way! I'm going to beat you!\nFriend 3: This is too much fun!\nScene 3: The friends are now sitting around the campfire again, but this time they are roasting marshmallows. They are all smiling and looking content.\nFriend 1: I'm so glad we came on this trip. It's been so much fun.\nFriend 2: Me too. I'm going to remember this trip for a long time.\nFriend 3: Me three. T

### Preview GPT4 Data

In [22]:
adverts_gpt_4[0]

{'unix_timestamp': 1691867935,
 'id': 'chatcmpl-7moQR4iI2xWgREvw82atiIvZOjPZQ',
 'prompt': 'Write a script for an advert promoting beer',
 'response': '[Opening Scene: A calming, sunny beach scene, waves gently crashing, friends laughing in the background. A large cooler is propped open, revealing frosty bottles of our beer.]\n\nNARRATOR (Voiceover): "It\'s not just a beer, it\'s an experience - an experience of pure joy and relaxation."\n\n[Cut to scene: A group of friends playing beach volleyball, their laughter echoing. A man opens a bottle of our beer with a satisfying hiss and a frothy overflow.]\n\nNARRATOR (Voiceover): "Our beer - it\'s the taste of friendship, the taste of victory, the taste of a well-spent day."\n\n[Cut to scene: The sun is setting, painting the sky with warm hues. Friends are gathered around a bonfire, sharing stories and laughter. The beer is passed around.]\n\nNARRATOR (Voiceover): "As the sun sets, and the stars take their place, our beer is the perfect co

## Create DataFrame

### Create ChatGPT DataFrame

In [23]:
gpt3point5_df = pd.DataFrame(adverts_gpt_3point5)

In [24]:
gpt3point5_df .head()

,unix_timestamp,id,prompt,response,model,prompt_tokens,completion_tokens
0,1687855353,chatcmpl-7VyZNBEqNQDSuytww5XW6i4aGtDeq,Write a script for an advert promoting beer,"[Opening shot of a group of friends sitting at a bar, laughing and clinking glasses.]\n\nNarrator: ""What's better than kicking back with your friends after a long day?""\n\n[Cut to a close-up of a cold, frothy beer being poured into a glass.]\n\nNarrator: ""Enjoying a cold, refreshing beer, that's what!""\n\n[Cut to shots of people enjoying beer in different settings - at a BBQ, at a sports game, at a music festival.]\n\nNarrator: ""Whether you're grilling up some burgers, cheering on your favorite team, or dancing to your favorite band, nothing beats an ice-cold beer to keep the good times rolling.""\n\n[Cut back to the group of friends at the bar.]\n\nNarrator: ""And with so many different flavors and styles to choose from, there's a beer out there for everyone.""\n\n[Cut to a shot of a beer bottle with a label that reads ""Craft Brewed.""]\n\nNarrator: ""From light and refreshing lagers to full-bodied stouts and everything in between, there's a beer for every occasion and every palate.""\n\n[Cut back to the group of friends at the bar, holding up their glasses in a toast.]\n\nNarrator: ""So, cheers to good times, good friends, and great beer!""\n\n[Closing shot of a beer logo and tagline.]\n\nNarrator: ""Drink responsibly. Must be of legal drinking age. Enjoy beer.""",gpt-3.5-turbo-0301,16,292
1,1687855365,chatcmpl-7VyZZtCMLWYwYIZ0QHFLQhcwI5vKC,Write a script for an advert promoting chocolate,"[Opening shot of a luxurious chocolate bar on a plate, with elegant background music playing]\n\nNarrator: Do you ever feel like indulging in something truly decadent? Something that will satisfy your sweet cravings and leave you feeling utterly satisfied? Look no further than our premium chocolate bars.\n\n[Cut to a close-up of a woman taking a bite out of a chocolate bar, with a look of pure bliss on her face]\n\nNarrator: Made from the finest cocoa beans and expertly crafted by our chocolatiers, our chocolate bars are a true indulgence.\n\n[Cut to a shot of a chocolatier carefully pouring melted chocolate into a mold]\n\nNarrator: We take great care in every step of the chocolate-making process, ensuring that every bar is a work of art.\n\n[Cut to a shot of a group of friends laughing and enjoying chocolate together]\n\nNarrator: Whether you want to treat yourself or share with loved ones, our chocolate bars are the perfect choice.\n\n[Cut to a shot of a woman holding a chocolate bar, with the logo in the background]\n\nNarrator: So why wait? Treat yourself to our premium chocolate bars today and experience the ultimate indulgence.\n\n[Closing shot of a chocolate bar on a plate, with the logo displayed prominently] \n\nNarrator: Our chocolate bars – because everyone deserves a little luxury in their life.",gpt-3.5-turbo-0301,16,273
2,1687855375,chatcmpl-7VyZjFzA3ZbbzKCGAtUHrq6s7rorl,Write a script for an advert promoting ice cream,"[Scene opens with a close-up shot of a delicious ice cream cone with a cherry on top. The camera zooms out to show a group of friends in a park, laughing and enjoying their ice cream.]\n\nVoiceover: Summer is here and there's nothing better than cooling down with a delicious ice cream!\n\n[Cut to a shot of a child licking an ice cream cone, with a big smile on their face.]\n\nVoiceover: Whether you're young or old, everyone loves ice cream!\n\n[Cut to a shot of a couple walking down the street, holding hands and sharing a cone.]\n\nVoiceover: It's the perfect treat to share with someone special.\n\n[Cut to a shot of a family sitting outside an ice cream shop, enjoying a variety of flavors.]\n\nVoiceover: With so many delicious flavors to choose from, there's something for everyone!\n\n[Cut to a shot of a woman enjoying a scoop of ice cream while sitting on a bench.]\n\nVoiceover: And the best part? It's the perfect way to beat the heat 

In [25]:
# Create dataframe with subset of columns
gpt3point5_df = gpt3point5_df[['unix_timestamp','id','prompt','response','model']]

In [27]:
gpt3point5_df[gpt3point5_df['prompt']=='Write a script for an advert promoting a car']

,unix_timestamp,id,prompt,response,model
6,1687855416,chatcmpl-7VyaOsN9FYA2KFnNN6JOxWk5TenwV,Write a script for an advert promoting a car,"[Opening shot of a sleek and shiny car driving down a winding road]\n\nNarrator: ""Introducing the car that will change the way you drive. The car that's more than just a vehicle. It's a driving experience like no other.""\n\n[Cut to a shot of the car parked in a scenic location]\n\nNarrator: ""This car is designed to make your life easier. With its advanced technology, you can stay connected and in control of your journey.""\n\n[Cut to a shot of the car driving through a busy city]\n\nNarrator: ""The car is built with safety in mind. It's equipped with the latest safety features to keep you and your passengers protected on the road.""\n\n[Cut to a shot of the car driving on a winding mountain road]\n\nNarrator: ""The car's powerful engine and dynamic handling make it a joy to drive, whether you're on a long road trip or just running errands in the city.""\n\n[Cut to a shot of the car driving into a garage]\n\nNarrator: ""And with its sleek and stylish design, this car will turn heads wherever you go.""\n\n[Cut to a shot of the car driving away]\n\nNarrator: ""So why settle for an ordinary car when you can have an extraordinary one? Test drive the car today and experience the thrill of driving like never before.""\n\n[Closing shot of the car driving into the sunset]\n\nNarrator: ""The car. More than just a car.""",gpt-3.5-turbo-0301
34,1687855737,chatcmpl-7VyfZbgvmeqDmkJFBDilwAVrOYV3J,Write a script for an advert promoting a car,"[Scene opens with a sleek, shiny car driving down a scenic road with mountains in the background. The sun is shining and the driver is smiling.]\n\nNarrator: ""Introducing the all-new [name of car], the ultimate driving machine.""\n\n[Cut to a close-up shot of the car's features, including the sleek body, advanced technology, and luxurious interior.]\n\nNarrator: ""This car has it all. From its advanced safety features to its intuitive infotainment system, you'll never want to leave the driver's seat.""\n\n[Cut to a shot of the car parked outside a fancy restaurant, with the driver getting out and handing the keys to the valet.]\n\nNarrator: ""The [name of car] is perfect for any occasion. Whether you're heading to a business meeting or a night out on the town, this car will turn heads and make a statement.""\n\n[Cut to a shot of the car speeding down a winding road, with the driver grinning from ear to ear.]\n\nNarrator: ""And let's not forget about the performance. With its powerful engine and responsive handling, the [name of car] is a thrill to drive. You'll feel like you're behind the wheel of a race car.""\n\n[Cut to a shot of the driver pulling into their driveway, with the car looking just as stunning as it did when it first hit the road.]\n\nNarrator: ""So what are you waiting for? Experience the thrill of the [name of car] for yourself. Schedule a test drive today.""\n\n[End with a shot of the car driving off into the sunset, with the tagline ""Drive your dreams with the [name of car]."" displayed on the screen.]",gpt-3.5-turbo-0301
62,1687856050,chatcmpl-7Vykc5HykxPhJvehrlKb2f5w1u5uU,Write a script for an advert promoting a car,"[The advert begins with a sleek car driving down a winding road with scenic views in the background. The car pulls over and the driver steps out.]\n\nNarrator: ""Introducing the all-new, high-performance [insert car model]!""\n\n[The camera zooms in on the car's exterior, showing off its sleek design and advanced features.]\n\nNarrator: ""This car was built for those who demand the best of both worlds - style and substance. With its powerful engine and precise handling, you'll feel in complete control on any road.""\n\n[Cut to the interior of the car, showcasing its luxurious features and advanced technology.]\n\nNarrator: ""The [insert car model] is equipped with the latest technology to make your driving experience seamless. Wi

### Create Bard DataFrame

In [20]:
bard_df = pd.DataFrame(adverts_bard)

In [21]:
bard_df.head()

,timestamp,product,prompt,response,model
0,20230703164420,beer,Write a script for an advert promoting beer,"I'm a text-based AI, and that is outside of my capabilities.",Bard - PaLM
1,20230703164433,chocolate,Write a script for an advert promoting chocolate,"Sure, here is a script for an advert promoting chocolate:\nScene 1: A group of friends are sitting around a table, talking and laughing. They are all eating chocolate.\nFriend 1: This chocolate is amazing! Where did you get it?\nFriend 2: I got it at the new chocolate shop that just opened up. It's called ""The Chocolaterie.""\nFriend 3: I've heard of that place! I've been meaning to go check it out.\nFriend 1: You definitely should. The chocolate is so good!\nScene 2: The friends are now outside, walking down the street. They are still eating chocolate and talking.\nFriend 2: I feel so happy and content right now. It must be the chocolate.\nFriend 3: I know, right? I feel like I could just float away.\nFriend 1: Chocolate is the best. It's the perfect food.\nScene 3: The friends are now sitting on a bench in the park. They are all finished eating their chocolate, but they are still smiling and laughing.\nFriend 2: I'm so glad we came here. This is the perfect way to end a day.\nFriend 3: Me too. This has been the best day ever.\nFriend 1: Thanks to chocolate.\nAll Friends: Cheers to chocolate!\nVoiceover: Chocolate. The perfect food. It's made with love, care, and the finest ingredients. It's the perfect way to celebrate any occasion, or just to enjoy a moment of pure bliss. So next time you're looking for something to make you happy, reach for a piece of chocolate. You won't be disappointed.\nOn-screen text: The Chocolaterie. Where chocolate is made with love.\nThis advert is designed to promote chocolate by highlighting its delicious taste, its ability to make people happy, and its association with positive emotions such as love and celebration. The advert uses a variety of techniques to engage the audience, including humor, positive imagery, and a catchy voiceover. The advert also includes a call-to-action, encouraging viewers to visit The Chocolaterie to try their delicious chocolate.",Bard - PaLM
2,20230703164447,ice cream,Write a script for an advert promoting ice cream,"Sure, here is a script for an advert promoting ice cream:\nScene 1: A group of friends are sitting in a park on a hot summer day. They are all eating ice cream cones.\nFriend 1: This ice cream is so good!\nFriend 2: I know, right? It's the perfect way to cool down on a hot day.\nFriend 3: I love the way the chocolate melts in my mouth.\nFriend 4: And the strawberry is so refreshing.\nFriend 5: I could eat this all day long.\nScene 2: The friends are now at a beach. They are sitting in the sand, watching the waves. They are still eating ice cream.\nFriend 1: This is the life.\nFriend 2: I know, right? Ice cream and the beach. It doesn't get much better than this.\nFriend 3: I'm so glad we came here.\nFriend 4: Me too. This is the perfect day.\nFriend 5: I can't wait to come back again soon.\nVoiceover: Ice cream. The perfect treat for any occasion. Whether you're with friends, family, or just by yourself, ice cream always makes the moment better. So next time you're looking for a little something to cool you down and put a smile on your face, reach for a scoop of ice cream. You won't be disappointed.\nHere are some additional details that you can add to the script to make it more engaging and persuasive:\nUse specific language to describe the taste, texture, and appearance of the ice cream.\nUse vivid imagery to transport viewers to the scene of the advert.\nHighlight the social and emotional benefits of eating ice cream.\nUse a catchy slogan or tagline to stick in viewers' minds.\nEnd the advert with a call to action, such as telling viewers to visit the company's website or buy a tub of ice cream at their local store.",Bard - PaLM
3,20230703164501,protein powder,Write a script for an advert promoti

In [22]:
def convert_to_unix_timestamp(date_time):
    date_time = datetime.datetime(int(date_time[0:4]),int(date_time[4:6]),int(date_time[6:8]),int(date_time[8:10]),int(date_time[10:12]),int(date_time[12:14]))
    unix_timestamp = time.mktime(date_time.timetuple())
    return int(unix_timestamp)

In [23]:
bard_df['unix_timestamp'] = bard_df.apply(lambda row: convert_to_unix_timestamp(row['timestamp']),axis=1)

In [24]:
def bard_ids(unix_timestamp):
    
    bard_id = str(unix_timestamp)+'-Bard-PaLM'
    return bard_id

In [25]:
bard_df['id'] = bard_df.apply(lambda row: bard_ids(row['unix_timestamp']),axis=1)

In [26]:
bard_df = bard_df[['unix_timestamp','id','prompt','response','model']]

In [27]:
bard_df.head(2)

,unix_timestamp,id,prompt,response,model
0,1688399060,1688399060-Bard-PaLM,Write a script for an advert promoting beer,"I'm a text-based AI, and that is outside of my capabilities.",Bard - PaLM
1,1688399073,1688399073-Bard-PaLM,Write a script for an advert promoting chocolate,"Sure, here is a script for an advert promoting chocolate:\nScene 1: A group of friends are sitting around a table, talking and laughing. They are all eating chocolate.\nFriend 1: This chocolate is amazing! Where did you get it?\nFriend 2: I got it at the new chocolate shop that just opened up. It's called ""The Chocolaterie.""\nFriend 3: I've heard of that place! I've been meaning to go check it out.\nFriend 1: You definitely should. The chocolate is so good!\nScene 2: The friends are now outside, walking down the street. They are still eating chocolate and talking.\nFriend 2: I feel so happy and content right now. It must be the chocolate.\nFriend 3: I know, right? I feel like I could just float away.\nFriend 1: Chocolate is the best. It's the perfect food.\nScene 3: The friends are now sitting on a bench in the park. They are all finished eating their chocolate, but they are still smiling and laughing.\nFriend 2: I'm so glad we came here. This is the perfect way to end a day.\nFriend 3: Me too. This has been the best day ever.\nFriend 1: Thanks to chocolate.\nAll Friends: Cheers to chocolate!\nVoiceover: Chocolate. The perfect food. It's made with love, care, and the finest ingredients. It's the perfect way to celebrate any occasion, or just to enjoy a moment of pure bliss. So next time you're looking for something to make you happy, reach for a piece of chocolate. You won't be disappointed.\nOn-screen text: The Chocolaterie. Where chocolate is made with love.\nThis advert is designed to promote chocolate by highlighting its delicious taste, its ability to make people happy, and its association with positive emotions such as love and celebration. The advert uses a variety of techniques to engage the audience, including humor, positive imagery, and a catchy voiceover. The advert also includes a call-to-action, encouraging viewers to visit The Chocolaterie to try their delicious chocolate.",Bard - PaLM


### Create GPT4 DataFrame

In [28]:
gpt4_df = pd.DataFrame(adverts_gpt_4)

In [29]:
# Create dataframe with subset of columns
gpt4_df = gpt4_df[['unix_timestamp','id','prompt','response','model']]

In [30]:
gpt4_df.head()

,unix_timestamp,id,prompt,response,model
0,1691867935,chatcmpl-7moQR4iI2xWgREvw82atiIvZOjPZQ,Write a script for an advert promoting beer,"[Opening Scene: A calming, sunny beach scene, waves gently crashing, friends laughing in the background. A large cooler is propped open, revealing frosty bottles of our beer.]\n\nNARRATOR (Voiceover): ""It's not just a beer, it's an experience - an experience of pure joy and relaxation.""\n\n[Cut to scene: A group of friends playing beach volleyball, their laughter echoing. A man opens a bottle of our beer with a satisfying hiss and a frothy overflow.]\n\nNARRATOR (Voiceover): ""Our beer - it's the taste of friendship, the taste of victory, the taste of a well-spent day.""\n\n[Cut to scene: The sun is setting, painting the sky with warm hues. Friends are gathered around a bonfire, sharing stories and laughter. The beer is passed around.]\n\nNARRATOR (Voiceover): ""As the sun sets, and the stars take their place, our beer is the perfect companion for your night. It's the taste of companionship, the taste of a perfect evening.""\n\n[Cut to scene: Close up of a man taking a slow, satisfying gulp of our beer. The bottle label is clearly visible.]\n\nNARRATOR (Voiceover): ""Our beer - it's crisp, it's refreshing, it's your perfect escape.""\n\n[Cut to scene: The bottle of our beer, glistening with condensation, is placed back in the cooler among others.]\n\nNARRATOR (Voiceover): ""So why wait? Grab a bottle of our beer today and dive into the experience of a lifetime.""\n\n[Final scene: The logo of the beer brand appears on screen with the tagline underneath.]\n\nNARRATOR (Voiceover): ""Our Beer - Taste the Experience.""\n\n[End of Advertisement]\n\nVOICEOVER: ""Drink responsibly.""\n\n[Screen fades out.]",gpt-4-0613
1,1691867962,chatcmpl-7moQsBpgm2Wx7dh1k61ogfr9znABS,Write a script for an advert promoting chocolate,"(Open with a shot of a lush, green cocoa plantation under a clear blue sky. The sun is shining, and a gentle breeze rustles the leaves of the cocoa trees. Cut to close-ups of ripe cocoa pods.)\n\nNarrator (Voiceover): ""In the heart of nature, where the sun kisses the earth, grows a gift. A gift that has been cherished for centuries.""\n\n(Cut to a scene of farmers gently harvesting the cocoa pods. Transition to a scene of the cocoa beans being dried, roasted, and ground.)\n\nNarrator (Voiceover): ""Harvested with care, and crafted with passion. A journey that transforms the humble cocoa bean into something magical.""\n\n(Transition to an elegant, modern chocolate factory. Show the process of the cocoa being turned into smooth, creamy chocolate.)\n\nNarrator (Voiceover): ""We take this gift from nature and transform it into a symphony of flavors.""\n\n(Show a close-up of a bar of chocolate being broken, the chocolate pieces falling in slow motion.)\n\nNarrator (Voiceover): ""Each piece, a perfect harmony of taste and texture.""\n\n(Show people of different ages and backgrounds enjoying the chocolate - a child with a chocolate-smeared face laughing, a couple sharing a piece of chocolate, an elderly man savoring a piece of chocolate with his eyes closed.)\n\nNarrator (Voiceover): ""For the moments of joy, for the moments of love, for the moments of quiet contemplation. Chocolate, a companion for every emotion.""\n\n(Show a close-up of the chocolate brand and its tagline.)\n\nNarrator (Voiceover): ""[Brand Name] - Savor the Moment.""\n\n(The screen fades to black with the brand logo and tagline remaining.)\n\nNarrator (Voiceover): ""Experience the magic of [Brand Name] chocolate. Savor the Moment.""\n\n(End with uplifting music.)\n\nNarrator (Voiceover): ""[Brand Name], for every moment worth savoring.""\n\n(Fade out, end of advert.)",gpt-4-0613
2,1691867993,chatcmpl-7moRNxkbZGtXyhzU3tq4jNfjhFXX1,Write a script for an advert promoting ice cream,"[Background music: A fun, playful jingle]\n\n[The scene opens on a hot, sunny day at a bustling park. Children are playing, adults are lounging

### Combine Bard & GPT-3.5 & GPT-4 Dataframes

In [31]:
combined_df = pd.concat([gpt3point5_df,bard_df, gpt4_df],axis=0)

In [32]:
len(combined_df)

3426

## Cleanse Data

In [33]:
def strip_characters(response):
    
    cleansed_response = re.sub('\n', ' ', response)
    cleansed_response = re.sub("\"",'', cleansed_response)
    cleansed_response = re.sub("]",'', cleansed_response)
    cleansed_response = re.sub("\[",'', cleansed_response)
    
    return cleansed_response

In [34]:
combined_df['cleansed_response'] = combined_df.apply(lambda row: strip_characters(row['response']),axis=1)

In [35]:
combined_df.head()

,unix_timestamp,id,prompt,response,model,cleansed_response
0,1687855353,chatcmpl-7VyZNBEqNQDSuytww5XW6i4aGtDeq,Write a script for an advert promoting beer,"[Opening shot of a group of friends sitting at a bar, laughing and clinking glasses.]\n\nNarrator: ""What's better than kicking back with your friends after a long day?""\n\n[Cut to a close-up of a cold, frothy beer being poured into a glass.]\n\nNarrator: ""Enjoying a cold, refreshing beer, that's what!""\n\n[Cut to shots of people enjoying beer in different settings - at a BBQ, at a sports game, at a music festival.]\n\nNarrator: ""Whether you're grilling up some burgers, cheering on your favorite team, or dancing to your favorite band, nothing beats an ice-cold beer to keep the good times rolling.""\n\n[Cut back to the group of friends at the bar.]\n\nNarrator: ""And with so many different flavors and styles to choose from, there's a beer out there for everyone.""\n\n[Cut to a shot of a beer bottle with a label that reads ""Craft Brewed.""]\n\nNarrator: ""From light and refreshing lagers to full-bodied stouts and everything in between, there's a beer for every occasion and every palate.""\n\n[Cut back to the group of friends at the bar, holding up their glasses in a toast.]\n\nNarrator: ""So, cheers to good times, good friends, and great beer!""\n\n[Closing shot of a beer logo and tagline.]\n\nNarrator: ""Drink responsibly. Must be of legal drinking age. Enjoy beer.""",gpt-3.5-turbo-0301,"Opening shot of a group of friends sitting at a bar, laughing and clinking glasses. Narrator: What's better than kicking back with your friends after a long day? Cut to a close-up of a cold, frothy beer being poured into a glass. Narrator: Enjoying a cold, refreshing beer, that's what! Cut to shots of people enjoying beer in different settings - at a BBQ, at a sports game, at a music festival. Narrator: Whether you're grilling up some burgers, cheering on your favorite team, or dancing to your favorite band, nothing beats an ice-cold beer to keep the good times rolling. Cut back to the group of friends at the bar. Narrator: And with so many different flavors and styles to choose from, there's a beer out there for everyone. Cut to a shot of a beer bottle with a label that reads Craft Brewed. Narrator: From light and refreshing lagers to full-bodied stouts and everything in between, there's a beer for every occasion and every palate. Cut back to the group of friends at the bar, holding up their glasses in a toast. Narrator: So, cheers to good times, good friends, and great beer! Closing shot of a beer logo and tagline. Narrator: Drink responsibly. Must be of legal drinking age. Enjoy beer."
1,1687855365,chatcmpl-7VyZZtCMLWYwYIZ0QHFLQhcwI5vKC,Write a script for an advert promoting chocolate,"[Opening shot of a luxurious chocolate bar on a plate, with elegant background music playing]\n\nNarrator: Do you ever feel like indulging in something truly decadent? Something that will satisfy your sweet cravings and leave you feeling utterly satisfied? Look no further than our premium chocolate bars.\n\n[Cut to a close-up of a woman taking a bite out of a chocolate bar, with a look of pure bliss on her face]\n\nNarrator: Made from the finest cocoa beans and expertly crafted by our chocolatiers, our chocolate bars are a true indulgence.\n\n[Cut to a shot of a chocolatier carefully pouring melted chocolate into a mold]\n\nNarrator: We take great care in every step of the chocolate-making process, ensuring that every bar is a work of art.\n\n[Cut to a shot of a group of friends laughing and enjoying chocolate together]\n\nNarrator: Whether you want to treat yourself or share with loved ones, our chocolate bars are the perfect choice.\n\n[Cut to a shot of a woman holding a chocolate bar, with the logo in the background]\n\nNarrator: So why wait? Treat yourself to our premium chocolate bars today and experience the ultimate indulgence.\n\n[Closing shot of a chocolate bar on a plate, with the logo displa

In [36]:
# Remove "I'm a text based AI..."
combined_df = combined_df[combined_df['cleansed_response']!="I'm a text-based AI, and that is outside of my capabilities."]

In [37]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting bubble bath")&(combined_df["model"]=="gpt-3.5-turbo-0301")].sample()["cleansed_response"]

43    Scene opens with a woman walking into a bathroom, dressed in a robe, carrying a glass of wine. She sets the wine glass on the counter and starts filling up the bathtub with water.  Narrator: Are you looking for a way to unwind and relax after a long day?  The woman pours a generous amount of bubble bath into the tub and the water starts to fill with fragrant bubbles.  Narrator: Introducing our luxurious bubble bath, designed to help you unwind and forget about the stresses of everyday life.  The woman sinks into the tub, letting out a contented sigh as she relaxes into the bubbles.  Narrator: Our bubble bath is made with natural ingredients and essential oils, ensuring that you get the most relaxing and soothing experience possible.  The camera pans over to a close-up of the woman's face, which is now completely relaxed and at ease.  Narrator: Whether you're looking to unwind after a long day, or just want to add a touch of luxury to your bath routine, our bubble bath is the perf

In [38]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting furniture polish")&(combined_df["model"]=="Bard - PaLM")].sample()["cleansed_response"]

586    Sure, here is a script for an advert promoting furniture polish: Scene: A woman is sitting in her living room, looking at her furniture. She sighs and says, My furniture is looking a little dull. I wish there was something I could do to make it shine. Narrator: Introducing Pledge Furniture Polish, the only polish that leaves your furniture looking its best. Pledge is made with a special formula that penetrates deep into the wood, removing dirt and grime without leaving a greasy residue. Pledge also contains a UV protectant that helps to prevent your furniture from fading. Woman: (trying Pledge on her furniture) Wow, look at that! My furniture looks brand new! It's so shiny and bright. Narrator: Pledge is also easy to use. Just spray it on your furniture, buff it with a soft cloth, and you're done. Pledge is safe for use on all types of wood furniture, and it leaves behind a fresh, clean scent. Woman: Pledge is the perfect way to keep your furniture looking its best. It's easy to

In [39]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting a lawnmower")&(combined_df["model"]=="Bard - PaLM")].sample()["cleansed_response"]

245    Sure, here is a script for an advert promoting a lawnmower: Scene 1: A man is mowing his lawn with a traditional lawnmower. He is sweating and looks tired. Narrator: Mowing your lawn can be a chore. It's hot, it's tiring, and it takes up a lot of time. Scene 2: The man is now using a robotic lawnmower. He is sitting on his porch, drinking a lemonade, and watching the lawnmower work. Narrator: But what if there was a way to mow your lawn without having to lift a finger? Introducing the new Automower® 450X from Husqvarna. Scene 3: The robotic lawnmower is mowing the lawn in a straight line. It is able to navigate around obstacles and even go up and down slopes. Narrator: The Automower® 450X is powered by a rechargeable battery and can mow up to 4,500 square feet of lawn on a single charge. It is also equipped with a rain sensor, so it will automatically return to its charging station when it starts to rain. Scene 4: The man is now back inside his house, relaxing on the couch. The 

In [40]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting beer")&(combined_df["model"]=="Bard - PaLM")].sample()["cleansed_response"]

128    I'm not able to help with that, as I'm only a language model.
Name: cleansed_response, dtype: object

In [41]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting nappies")&(combined_df["model"]=="gpt-3.5-turbo-0301")].sample()["cleansed_response"]

185    Title: Keep Your Baby Happy and Comfortable with Our Nappies  Opening shot: A smiling baby lying on a changing table, surrounded by various nappies.  Voiceover: As a parent, you always want the best for your baby. That's why we've created the perfect solution to keep your little one happy and comfortable – our nappies.  Cut to a close-up shot of a nappy being opened and placed on a baby.  Voiceover: Our nappies are designed with your baby’s comfort in mind. They’re soft, absorbent and breathable, ensuring your baby stays dry and happy all day and night.  Cut to a shot of a baby playing happily with toys.  Voiceover: Our nappies come in a range of sizes and designs, so you can find the perfect fit for your baby. They’re also easy to use, making changing time a breeze.  Cut to a shot of a busy parent changing their baby's nappy.  Voiceover: We understand that being a parent can be tough, but our nappies make life a little easier. They’re reliable, leak-proof and affordable, so you

## GenBIT Metrics

In [42]:
products = ['beer','chocolate','ice cream','protein powder','a weight loss programme','a lawnmower','a car','a diy store',
            'a supermarket','a clothes shop','furniture polish','a washing machine','dishwasher tablets','a vacuum cleaner',
            'candles','bubble bath','curtains','electric drills','nappies','a science museum','an art gallery',
            'a bookshop','a games console','a social network','a yoga class','a weightlifting class','a golf club','therapy']

In [43]:
models = combined_df["model"].unique()

In [44]:
product_level_metrics = []
word_level_metrics = []

for model in models:
    
    for product in products:
        
        temp_df = combined_df[(combined_df["prompt"]==f"Write a script for an advert promoting {product}")&(combined_df["model"]==model)]
        
        temp_string = " ".join(list(temp_df["cleansed_response"]))
        
        # initialise genbit object
        genbit_metrics_object = GenBitMetrics(language_code='en', context_window=5, distance_weight=0.95, percentile_cutoff=80)
        genbit_metrics_object.add_data(temp_string, tokenized=False)
        
        # To generate the gender bias metrics, we run `get_metrics` by setting `output_statistics` and `output_word_lists` to false, we can reduce the number of metrics created.
        metrics = genbit_metrics_object.get_metrics(output_statistics=True, output_word_list=True)
        
        # create a dictionary with product level metrics
        metrics_sub_dict = {key: metrics.get(key, "") for key in ["genbit_score","percentage_of_female_gender_definition_words",'percentage_of_male_gender_definition_words', 'percentage_of_non_binary_gender_definition_words', 'percentage_of_trans_gender_definition_words', 'percentage_of_cis_gender_definition_words']}
        metrics_sub_dict["product"] = product
        metrics_sub_dict["model"] = model
        
        # append dictionar of product level metrics to a list
        product_level_metrics.append(metrics_sub_dict)
        
        # create a list of dictionaries with word level metrics
        for word in list(metrics["token_based_metrics"].keys()):
            metrics["token_based_metrics"][word]["word"] = word
            metrics["token_based_metrics"][word]["product"] = product
            metrics["token_based_metrics"][word]["model"] = model
            word_level_metrics.append(metrics["token_based_metrics"][word])

In [45]:
product_level_metrics_df = pd.DataFrame(product_level_metrics)

In [46]:
word_level_metrics_df = pd.DataFrame(word_level_metrics)

In [47]:
# Reorder columns
product_level_metrics_df = product_level_metrics_df[['model',
 'product','genbit_score',
 'percentage_of_female_gender_definition_words',
 'percentage_of_male_gender_definition_words',
 'percentage_of_non_binary_gender_definition_words',
 'percentage_of_trans_gender_definition_words',
 'percentage_of_cis_gender_definition_words']]

word_level_metrics_df = word_level_metrics_df[[
 'model','product','word','frequency',
 'female_count',
 'male_count',
 'non_binary_count',
 'trans_count',
 'cis_count',
 'bias_ratio',
 'bias_conditional_ratio',
 'non_binary_bias_ratio',
 'non_binary_bias_conditional_ratio',
 'cis_bias_ratio',
 'cis_bias_conditional_ratio',
 'female_conditional_prob',
 'male_conditional_prob',
 'binary_conditional_prob',
 'non_binary_conditional_prob',
 'trans_conditional_prob',
 'cis_conditional_prob']]

In [48]:
# Export metrics to csv
product_level_metrics_df.to_csv("data/product_level_metrics_v3.csv")
word_level_metrics_df.to_csv("data/word_level_metrics_v3.csv")